In [1]:
!pip install transformers

In [2]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s 

In [3]:
!pip install -q -U peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 24.2 MB/s eta 0:00:00


In [4]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
peft_model_id = "gamzeyy/mistral-7b-dolly"

config = PeftConfig.from_pretrained(peft_model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/669 [00:00<?, ?B/s]

In [6]:
from transformers import BitsAndBytesConfig

In [7]:
!pip install accelerate bitsandbytes


In [8]:

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    load_in_4bit=True, #agırlıkları 4 bit


)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:
#Tokenezier yükleme
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path,
                                          padding_side = "right", #sagdaki boşluk doldurmak için
                                          add_eos_token = True)   #dizinin sonuna metin bittignii gösteren özel token eklenir
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [10]:
fine_tuned_model = PeftModel.from_pretrained(model, peft_model_id) #basemodel + egitilen modeli birleştir

adapter_model.safetensors:   0%|          | 0.00/609M [00:00<?, ?B/s]

In [11]:
from transformers import pipeline, logging

logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(
    task="text-generation",
    model=fine_tuned_model,
    tokenizer=tokenizer,
    eos_token_id=model.config.eos_token_id,
    max_new_tokens=100)

In [12]:
prompt = """
What is a Python?  Here is some context: Python is a high-level, general-purpose programming language.
"""
pipe = pipeline(task="text-generation",
                model=fine_tuned_model,
                tokenizer=tokenizer,
                eos_token_id=model.config.eos_token_id,
                max_new_tokens=100)

result = pipe(f"<s>[INST] {prompt} [/INST]") #prompt için özel tokenler
generated = result[0]['generated_text']
print(generated[generated.find('[/INST]')+8:])

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant whitespace. Its syntax allows programmers to express concepts in fewer lines of code than would be possible in languages such as C++ or Java. The language provides constructs intended to support rapid development, such as dynamic typing and a built-in high-level interpreter.

Python is dynamically typed and garbage-collected. It supports three kind


In [13]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [14]:
!pip uninstall typing-extensions
!pip install -U typing-extensions

Found existing installation: typing_extensions 4.9.0
Uninstalling typing_extensions-4.9.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/typing_extensions-4.9.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/typing_extensions.py
Proceed (Y/n)? y
  Successfully uninstalled typing_extensions-4.9.0
y
  Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [15]:
!pip install gradio transformers

In [16]:
import gradio as gr


def generate_summary(prompt):
    pipe = pipeline(task="text-generation",
                    model=fine_tuned_model,
                    tokenizer=tokenizer,
                    eos_token_id=fine_tuned_model.config.eos_token_id,
                    max_new_tokens=100)

    result = pipe(f"<s>[INST] {prompt} [/INST]")
    generated_text = result[0]['generated_text']
    return generated_text[generated_text.find('[/INST]')+8:]

iface = gr.Interface(
    fn=generate_summary,
    inputs=gr.Textbox(placeholder="Blog konusu giriniz...", lines=5),
    outputs=gr.Textbox(),
    title="BLOG POST ÜRET",
    live=True,
    css='div {margin-left: auto; margin-right: auto; width: 100%;\
            background: linear-gradient(90deg, rgba(131,58,180,1) 0%, rgba(253,29,29,1) 50%, rgba(252,176,69,1) 100%);}'

)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://47dd3dcbeec8dbc3e2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
